In [ ]:
import os
import json
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
def get_video_dimensions_and_fps(video_input_path):
    """Get video width, height, and frames per second (fps)."""
    cap = cv2.VideoCapture(video_input_path)
    if not cap.isOpened():
        raise IOError("Error: Could not open video file")

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()

    return width, height, fps

def extract_coco_points(path, width, height, subject_id):
    """Extract COCO keypoints from a JSON file and return a structured list."""
    json_path = os.path.join(path, "json", "keypoints_coco.json")
    with open(json_path, 'r') as f:
        keypoints_data = json.load(f)

    skeleton_id = -1
    frame_id = 0
    result = []

    for row in keypoints_data:
        if row['image_id'] == frame_id:
            skeleton_id += 1
        else:
            skeleton_id = 0
        frame_id = row['image_id']

        kpts = np.reshape(row['keypoints'], (-1, 3))[:, :2]
        result.append([subject_id, frame_id, skeleton_id, width, height, kpts])

    return result

def transpose_coordinates(df):
    """Transpose dataframe so each frame has one row with up to 3 coordinate sets."""
    transposed_data = {'frame': [], 'coordinates_0': [], 'coordinates_1': [], 'coordinates_2': []}

    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        frame = row['frame']
        coords = row['coordinates']

        if frame not in transposed_data['frame']:
            transposed_data['frame'].append(frame)
            transposed_data['coordinates_0'].append(coords)
            transposed_data['coordinates_1'].append(None)
            transposed_data['coordinates_2'].append(None)
        else:
            idx = transposed_data['frame'].index(frame)
            if transposed_data['coordinates_1'][idx] is None:
                transposed_data['coordinates_1'][idx] = coords
            elif transposed_data['coordinates_2'][idx] is None:
                transposed_data['coordinates_2'][idx] = coords

    return pd.DataFrame(transposed_data)

def remove_invalid_coordinates(df):
    """Replace coordinates with NaN if at least 8 points are [0.0, 0.0]."""
    coordinate_columns = ['coordinates_0', 'coordinates_1', 'coordinates_2']
    for column in coordinate_columns:
        df[column] = df[column].apply(lambda x: np.nan if x is None else x)

    for col in coordinate_columns:
        for index in tqdm(range(len(df))):
            coords = df[col].iloc[index]
            if isinstance(coords, float) and np.isnan(coords):
                continue
            zero_count = np.sum(np.all(coords == [0.0, 0.0], axis=1))
            if zero_count >= 8:
                df.at[index, col] = np.nan
    return df

def fill_missing_frames(df):
    """Fill in missing frames with NaN rows to ensure continuous frame indexing."""
    df = df.set_index('frame')
    all_frames = range(df.index.min(), df.index.max() + 1)
    df_filled = df.reindex(all_frames).reset_index()
    return df_filled

def main():
    # === USER INPUTS ===
    video_input_path = "path/to/video.mp4"
    json_folder_path = "path/to/json_folder"
    participant_id = 1234  # Change this to your subject ID

    # === GET VIDEO DIMENSIONS ===
    width, height, fps = get_video_dimensions_and_fps(video_input_path)
    print(f"Video dimensions: {width}x{height}")
    print(f"Frames per second (fps): {fps}")

    # === EXTRACT COCO KEYPOINTS ===
    coco_data = extract_coco_points(json_folder_path, width, height, participant_id)
    df = pd.DataFrame(coco_data, columns=["ppt", "frame", "orig_id", "width", "height", "coordinates"])
    df = df[["frame", "coordinates"]]

    # === TRANSPOSE COORDINATES ===
    df = transpose_coordinates(df)

    # === REMOVE INVALID ENTRIES ===
    df = remove_invalid_coordinates(df)

    # === FILL MISSING FRAMES ===
    df = fill_missing_frames(df)

    # === SAVE FINAL DATA ===
    output_path = f"{participant_id}_df.pkl"
    df[["coordinates_0", "coordinates_1", "coordinates_2"]].to_pickle(output_path)
    print(f"\nData saved to {output_path}")

if __name__ == "__main__":
    main()
